# Un esempio completo: Iris Dataset
Utilizziamo la nostra rete per un esempio di classificazione reale.<br>
L'<a href="https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html">Iris Dataset</a> consiste di 150 elementi di 4 features inerenti a 3 differenti tipologie di piante: Setosa, Versicolour, e Virginica.<br>
Le 4 features corrispondono a 4 caratteristiche della pianta: sepal length (cm), sepal width (cm), petal length (cm) e petal width (cm).<br>
Il nostro scopo sarà predire, a partire dalla caratteristica della pianta, la tiplogia di pianta.

In [1]:
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

<a href="https://scikit-learn.org/stable/">Scikit-Learn</a> offre il metodo <a href="https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html">load_iris</a> che consente di ottenere una in maniera semplice ed efficace il set di dati dell'Iris Dataset.<br>

In [2]:
# Load dati del dataset
iris_dataset = datasets.load_iris()

# Creazione Dataframe.
# Il dataframe viene creato per lavorare più agevolmente con i dati.
df = pd.DataFrame(iris_dataset.data, columns=iris_dataset.feature_names)
df['y'] = iris_dataset.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),y
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


Il dataframe può essere diviso in:
- X: contiene gli input della rete (features)
- y: per ogni input contiene l'etichetta di classe (label)<br>

A sua volta X e y possono essere divisi in:
- Set di Train: utile per addestrare la rete
- Set di Test: utile per testare la qualità della rete

In [3]:
# Divisione features e lables
X, y = df.drop(['y'], axis=1).as_matrix().astype('float64'), df['y'].as_matrix().astype('int64')

# Divisione train e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=94)


Di seguito viene creata la classe NeuralNetwork utilizzando i moduli layer, functions e optimizer.<br>
I moduli in oggetto contengono esattamente le classi spiegate nei notebook in allegato a questo tutorial.<br>
Si precisa che la creazione della classe <i></i>:
- Rende più agevole la lettura del codice
- Rende possibile utilizzare la medesima rete sia per la fase di Train che per quella di Test
- Rende più leggibile la sequenza di addestramento (forward, calcolo loss e backward)

In [4]:
from neural_network import layers, functions, optimizers

class NeuralNetwork_8:
    def __init__(self, len_in, len_out):  
        '''
            Inizializzazione della rete.
            La rete è composta da:
            - Un layer di input
            - Un layer Hidden composto da 8 neuroni
            - Un layer di output
            
            Keyword arguments:
            len_in -- Numero di features in input
            len_out -- Numero di classi in output
        
        '''
        self.dense1 = layers.Layer_Dense( len_in , 8 ) 
        self.relu = functions.Activation_ReLU()  
        self.dense2 =layers.Layer_Dense( 8 , len_out)
        self.softmax = functions.Activation_Softmax() 

        self.loss_activation = functions.Activation_Softmax_Loss_CategoricalCrossentropy()

        self.optimizer = optimizers.Optimizer_SGD(0.1)
        
    def forward(self, X): 
        '''
            Implementazione dello step di Forward
            
            Keyword arguments:
            X -- Batch in input alla rete
            
        '''
        # Chiamata agli step di forward in cascata (dall'input all'output)
        self.dense1.forward(X)
        self.relu.forward(self.dense1.output)
        self.dense2.forward(self.relu.output) 
        self.softmax.forward(self.dense2.output) 
        return self.softmax.output
    
    def loss(self, y_true):
        '''
            Calcolo della Loss.
            La loss è calcolata rispetto alla ground truth e al valore predetto nello
            step di forward.
            
            Keyword arguments:
            y_true -- Valore della ground truth
            
        '''
        return self.loss_activation.forward(self.dense2.output, y_true)

    def backpropagation(self, y_true):
        '''
            Implementazione della backpropagation.
            La Backpropagation si compone degli step di:
            - Calcolo del gradiente
            - Aggiornamento dei pesi con lo Stocastic Gradient Descend
            
            Keyword arguments:
            y_true -- Valore della ground truth
            
        '''
         # Chiamata agli step di backward in cascata (dall'output all'input)
        self.loss_activation.backward(self.loss_activation.output, y_true)
        self.dense2.backward(self.loss_activation.dinputs)
        self.relu.backward(self.dense2.dinputs)
        self.dense1.backward(self.relu.dinputs)
        
        self.optimizer.update_params(self.dense1)
        self.optimizer.update_params(self.dense2)
    
    def predict(self, X): 
        '''
            A partire da un input X, restituisce i valori predetti dalla rete.
            In particolare, per ogni input, viene restituita la probabilità 
            d'appartenenza ad ogni etichetta di classe (otuptu della funzione Softmax)
            
            Keyword arguments:
            X -- Batch in input alla rete
            
        '''
        return self.forward(X)
    
    def predict_classes(self, X): 
        '''
            A partire da un input X, restituisce i valori predetti dalla rete.
            In particolare, per ogni input, viene restituita l'etichetta di classe.
            
            Keyword arguments:
            X -- Batch in input alla rete
        ''' 
        return np.argmax(self.forward(X), axis = 1 )

In [5]:
# Inizializzazione della neural network
n_features = X.shape[1]
n_classes = len(np.unique(y))

model = NeuralNetwork_8(n_features, n_classes)

Dopo aver inizializzato la rete, è possibile procedere con la fase di training. <br>
La fase di training è composta a diverse iterazioni (epoche):
- In ogni epoca vengono eseguiti gli step di forward, calcolo della loss e backpropagation
- Lo step di backpropagation effettua la discesa del gradiente, aggiornando i pesi dei pesi e dei bias dei layer dense della rete

In [6]:
epochs = 1000

# Train in loop
for epoch in range (0, epochs):
    model.forward(X_train)
    
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = model.loss(y_train)
    
   
    predictions = np.argmax(model.predict(X_train), axis = 1 )
    
    if len (y_train.shape) == 2 :
        y_train = np.argmax(y_train, axis = 1 )
        
    accuracy = np.mean(predictions == y_train)
    
    if not epoch % 100:
        print ( f'epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss :.3f}' )
        
    # Backpropagation pass
    model.backpropagation(y_train)
    
     

epoch: 0, acc: 0.325, loss: 1.153
epoch: 100, acc: 0.800, loss: 0.367
epoch: 200, acc: 0.883, loss: 0.253
epoch: 300, acc: 0.933, loss: 0.147
epoch: 400, acc: 0.950, loss: 0.099
epoch: 500, acc: 0.958, loss: 0.083
epoch: 600, acc: 0.958, loss: 0.074
epoch: 700, acc: 0.958, loss: 0.068
epoch: 800, acc: 0.967, loss: 0.063
epoch: 900, acc: 0.967, loss: 0.059


L'oggetto <i>model</i> è un modello addestrato che può essere utilizzato. <br>
Può essere utilizzato il set di test per valutare la bonta della rete.

In [7]:
y_pred = model.predict_classes(X_test)
y_true = y_test

In [8]:
print(f'Accuracy:\n{accuracy_score(y_true, y_pred):.2f}\n')
print(f'Confusion Matrix\n{confusion_matrix(y_true, y_pred)}\n')
print(classification_report(y_true, y_pred))

Accuracy:
0.93

Confusion Matrix
[[10  0  0]
 [ 0 11  1]
 [ 0  1  7]]

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        10
          1       0.92      0.92      0.92        12
          2       0.88      0.88      0.88         8

avg / total       0.93      0.93      0.93        30

